# Power-tracing smartcards using LEIA

This notebook uses [**pyecsca**](https://pyecsca.org) and a [LEIA board](https://github.com/h2lab/smartleia) to do simple (SPA-like) power-tracing of selected smartcard targets. It assumess the user has a PicoScope 6000 oscilloscope (though this can be replaced by any oscilloscope that **pyecsca** supports). Similarly, the LEIA board can be replaced by simple smartcard reader, though the user then need a separate way of triggering the scope. The code also assumes the ECTester applet is already installed on the target cards. Use a tool like [GlobalPlatformPro](https://github.com/martinpaljak/GlobalPlatformPro) to install it.

See the [pyecsca notebook](https://pyecsca.org/notebook/measurement.html) on measurement for more examples.

In [ ]:
from pyecsca.sca.target.ectester import ECTesterTargetLEIA, KeypairEnum, ParameterEnum, CurveEnum, KeyEnum, KeyClassEnum, KeyBuildEnum, KeyAgreementEnum, SignatureEnum, TransformationEnum
from pyecsca.ec.params import load_params_ectester, get_params
from pyecsca.sca.scope.picoscope_sdk import PS6000Scope
from pyecsca.sca.trace import Trace
from pyecsca.sca.trace.plot import plot_trace, plot_traces
from pyecsca.sca.scope import SampleType

import numpy as np
from time import sleep
from smartleia import LEIA, TriggerPoints

import holoviews as hv

hv.extension("bokeh")
%opts RGB [height=700, responsive=True]

In [ ]:
from pyecsca.sca.trace.sampling import downsample_max, downsample_average, downsample_decimate
from pyecsca.sca.trace.process import rolling_mean, recenter, absolute, threshold
from pyecsca.sca.trace.filter import filter_lowpass
from pyecsca.sca.trace.edit import pad, trim

Create the LEIA interface and ECTester target.

In [ ]:
sl = LEIA()
ectester = ECTesterTargetLEIA(sl)

## Initialize scope and card
Connect to the card and setup the scope/measurement parameters based on it.

Note that several smartcards really are the same model and thus we merged them:
 - N4 = N5 = N10
 - N6 = N7
 - N3 = N8

In [ ]:
ectester.connect()

In [ ]:
atr = ectester.atr
print(atr, atr.hex())
card_map = {
    "3bd518ff8191fe1fc38073c8211309": "A1",
    "3bb89600c00831fe45ffff1154305023006a": "I1",
    "3bfe1800008031fe45803180664090a5102e1083019000f2": "I2",
    "3bf81800ff8131fe454a434f507632343143": "N1",
    "3bf81300008131fe454a434f5076323431b7": "N2N9",
    "3b9495810146545601c4": "N4N10",
    "3bd518ff8191fe1fc38073c821100a": "N6N11",
    "3bf91300008131fe454a434f503234325233a2":"N8",
    "3b9c9580811f039067464a01005404f272fe00c0": "F1",
    "3b90958011fe6a": "F2F3",
    "3b9f95803fc7a08031e073fa21106300000083f09000bb": "S1S2",
    "3bf99600008131fe4553434537200e00202028":"G1",
    "3bfe1800008031fe4553434536302d43443038312d6e46a9": "G2",
    "3b959540ffae01030000":"E2",
}
card = card_map.get(atr.hex(), None)
print(card)

In [ ]:
scope = PS6000Scope()

In [ ]:
scope.open()

In [ ]:
print(scope.get_variant())
if card == "A1":
    # Athena IDProtect
    # 35M for keygen
    # 13M for ecdh
    actual_freq, n_samples = scope.setup_frequency(frequency=50_000_000, pretrig=0, posttrig=35_000_000)
    scope.setup_channel(channel="A", coupling="DC", range=1, offset=0, enable=True)
    scope.setup_channel(channel="B", coupling="DC_50", range=0.2, offset=-0.24, enable=True)
    scope.setup_trigger(channel="A", threshold=0.2, direction="rising", delay=0, timeout=5000, enable=True)
    scope.setup_capture(channel="B", enable=True)
elif card == "I1":
    # Infineon SECORA
    # 6M for keygen
    # 3M for ecdh
    actual_freq, n_samples = scope.setup_frequency(frequency=50_000_000, pretrig=0, posttrig=6_000_000)
    scope.setup_channel(channel="A", coupling="DC", range=1, offset=0, enable=True)
    scope.setup_channel(channel="B", coupling="DC_50", range=0.2, offset=-0.25, enable=True)
    scope.setup_trigger(channel="A", threshold=0.2, direction="rising", delay=0, timeout=5000, enable=True)
    scope.setup_capture(channel="B", enable=True)
elif card == "I2":
    # Infineon CJTOP SLJ 52GLA0890AL M84
    actual_freq, n_samples = scope.setup_frequency(frequency=50_000_000, pretrig=0, posttrig=15_000_000)
    scope.setup_channel(channel="A", coupling="DC", range=1, offset=0, enable=True)
    scope.setup_channel(channel="B", coupling="DC_50", range=0.1, offset=-0.15, enable=True)
    scope.setup_trigger(channel="A", threshold=0.2, direction="rising", delay=0, timeout=5000, enable=True)
    scope.setup_capture(channel="B", enable=True)
elif card == "N1":
    # NXP J3A081
    # 30M for keygen (first), then 10M for subsequent
    # 10M for ecdh
    actual_freq, n_samples = scope.setup_frequency(frequency=50_000_000, pretrig=0, posttrig=30_000_000)
    scope.setup_channel(channel="A", coupling="DC", range=1, offset=0, enable=True)
    scope.setup_channel(channel="B", coupling="DC_50", range=0.05, offset=-0.18, enable=True)
    scope.setup_trigger(channel="A", threshold=0.2, direction="rising", delay=0, timeout=5000, enable=True)
    scope.setup_capture(channel="B", enable=True)
elif card == "N2N9":
    # NXP JCOP v2.4.1R3
    # N9
    actual_freq, n_samples = scope.setup_frequency(frequency=625_000_000, pretrig=0, posttrig=500_000_000)
    scope.setup_channel(channel="A", coupling="DC", range=1, offset=0, enable=True)
    scope.setup_channel(channel="B", coupling="DC_50", range=0.05, offset=-0.280, enable=True)
    scope.setup_trigger(channel="A", threshold=0.2, direction="rising", delay=0, timeout=5000, enable=True)
    scope.setup_capture(channel="B", enable=True)
    # N2
    #actual_freq, n_samples = scope.setup_frequency(frequency=50_000_000, pretrig=0, posttrig=40_000_000)
    #scope.setup_channel(channel="A", coupling="DC", range=1, offset=0, enable=True)
    #scope.setup_channel(channel="B", coupling="DC_50", range=0.1, offset=-0.18, enable=True)
    #scope.setup_trigger(channel="A", threshold=0.2, direction="rising", delay=0, timeout=5000, enable=True)
    #scope.setup_capture(channel="B", enable=True)
elif card == "N4N10":
    # NXP J3H145
    # 15M for keygen
    # 10M for ecdh
    actual_freq, n_samples = scope.setup_frequency(frequency=50_000_000, pretrig=0, posttrig=5_000_000)
    scope.setup_channel(channel="A", coupling="DC", range=1, offset=0, enable=True)
    scope.setup_channel(channel="B", coupling="DC_50", range=0.2, offset=-0.160, enable=True)
    scope.setup_trigger(channel="A", threshold=0.2, direction="rising", delay=0, timeout=5000, enable=True)
    scope.setup_capture(channel="B", enable=True)
elif card == "N6N11":
    # NXP JCOP4
    # 3M for keygen
    # 3M for ECDH
    actual_freq, n_samples = scope.setup_frequency(frequency=50_000_000, pretrig=0, posttrig=3_000_000)
    scope.setup_channel(channel="A", coupling="DC", range=1, offset=0, enable=True)
    scope.setup_channel(channel="B", coupling="DC_50", range=0.2, offset=-0.170, enable=True)
    scope.setup_trigger(channel="A", threshold=0.2, direction="rising", delay=0, timeout=5000, enable=True)
    scope.setup_capture(channel="B", enable=True)
elif card == "F1":
    # Javacos A22 CR
    actual_freq, n_samples = scope.setup_frequency(frequency=50_000_000, pretrig=0, posttrig=3_000_000)
    scope.setup_channel(channel="A", coupling="DC", range=1, offset=0, enable=True)
    scope.setup_channel(channel="B", coupling="DC_50", range=0.2, offset=-0.170, enable=True)
    scope.setup_trigger(channel="A", threshold=0.2, direction="rising", delay=0, timeout=5000, enable=True)
    scope.setup_capture(channel="B", enable=True)
elif card == "F2F3":
    # Javacos JC30M48 CR
    actual_freq, n_samples = scope.setup_frequency(frequency=50_000_000, pretrig=0, posttrig=3_000_000)
    scope.setup_channel(channel="A", coupling="DC", range=1, offset=0, enable=True)
    scope.setup_channel(channel="B", coupling="DC_50", range=0.2, offset=-0.170, enable=True)
    scope.setup_trigger(channel="A", threshold=0.2, direction="rising", delay=0, timeout=5000, enable=True)
    scope.setup_capture(channel="B", enable=True)
else:
    print("Unkown card, set the appropriate parameters manually.")
print(actual_freq, n_samples)

In [ ]:
ectester.select_applet()

In [ ]:
ectester.info()

## Allocate
Allocate a keypair and the signature + key exchange objects on the card.

If any of these fail. Try running `ectester.cleanup()` which frees some memory on the card.

In [ ]:
ectester.allocate(KeypairEnum.KEYPAIR_LOCAL,
                  KeyBuildEnum.BUILD_KEYBUILDER | KeyBuildEnum.BUILD_KEYPAIR,
                  256,
                  KeyClassEnum.ALG_EC_FP)

In [ ]:
ectester.allocate_sig(SignatureEnum.ALG_ECDSA_SHA)

In [ ]:
ectester.allocate_ka(KeyAgreementEnum.ALG_EC_SVDP_DH)

## Set params

Set a curve (domain parameters) on the keypair on the card.

In [ ]:
params = get_params("secg", "secp256r1", "affine")

# or set parameters that will lead to EPA issues
# params = load_params_ectester("../countermeasures/countermeasures/tests/comb/cofactor256p18446744073709551617_smallgen_fakeorder.csv", "affine")

In [ ]:
ectester.set(KeypairEnum.KEYPAIR_LOCAL,
             CurveEnum.external,
             ParameterEnum.DOMAIN_FP,
             ECTesterTargetLEIA.encode_parameters(ParameterEnum.DOMAIN_FP, params))

Enable the LEIA trigger on APDU send.

In [ ]:
sl.set_trigger_strategy(1, point_list=[TriggerPoints.TRIG_PRE_SEND_APDU], delay=0)

## Generate
Generate a keypair and capture a trace of it.

In [ ]:
scope.arm()
sleep(2)

In [ ]:
ectester.generate(KeypairEnum.KEYPAIR_LOCAL)

In [ ]:
scope.capture(10000)

In [ ]:
trace_gen = scope.retrieve("B", SampleType.Volt)

In [ ]:
plot_trace(trace_gen)

In [ ]:
plot_trace(downsample_average(trace_gen, 1000))

## Or set key
Capture a trace of the set-key operation on the keypair. This is interesting as some precomputation may happen.

In [ ]:
priv = 0x3c984f3a459a6b8f1a5ece87a695d1b112b978024a9c56c1a12ade3500f29d8c
pub = params.curve.affine_multiply(params.generator, priv + 5)

In [ ]:
scope.arm()
sleep(2)

In [ ]:
ectester.set(KeypairEnum.KEYPAIR_LOCAL,
             CurveEnum.external,
             ParameterEnum.S | ParameterEnum.W,
             {**ECTesterTargetLEIA.encode_parameters(ParameterEnum.S, priv),
              **ECTesterTargetLEIA.encode_parameters(ParameterEnum.W, pub)})

In [ ]:
scope.capture(10000)

In [ ]:
trace_set = scope.retrieve("B", SampleType.Volt)

In [ ]:
plot_trace(trace_set)

## ECDSA
Perform an ECDSA signature and capture a trace of it.

In [ ]:
scope.arm()
sleep(2)

In [ ]:
resp = ectester.ecdsa_sign(KeypairEnum.KEYPAIR_LOCAL,
                           True,
                           SignatureEnum.ALG_ECDSA_SHA,
                           b"message")
resp

In [ ]:
scope.capture(10000)

In [ ]:
trace_ecdsa = scope.retrieve("B", SampleType.Volt)

In [ ]:
plot_trace(trace_ecdsa)

## ECDH
Perform ECDH and capture a trace of it.

In [ ]:
scope.arm()
sleep(2)

In [ ]:
ectester.ecdh_direct(KeypairEnum.KEYPAIR_LOCAL,
                     True,
                     TransformationEnum.NONE,
                     KeyAgreementEnum.ALG_EC_SVDP_DH,
                     bytes(params.generator))

In [ ]:
scope.capture(10000)

In [ ]:
trace_ecdh = scope.retrieve("B", SampleType.Volt)

In [ ]:
plot_trace(trace_ecdh)

## Cleanup
Cleanup some memory on the card, disconnect from it and the scope.

In [ ]:
ectester.cleanup()

In [ ]:
ectester.disconnect()

In [ ]:
scope.close()

## Misc
Some miscellaneous remains of previous experiments: 🪦

In [ ]:
def find_bumps(trace):
    ds = downsample_average(trace, 1000)
    ts = threshold(ds, 0.025)
    prev = 0
    previ = 0
    total = []
    big = []
    for i, sample in enumerate(ts.samples):
        if prev == 0 and sample == 1:
            dist = i - previ
            if dist > 2500 and total:
                big.append(total)
                total = []
            if dist > 500:
                l = [i]
                total.append(l)
            else:
                total[-1].append(i)
            previ = i
            prev = sample
        elif prev == 1 and sample == 0:
            prev = sample
    if total:
        big.append(total)
    s = []
    for t in big:
        seq = []
        for l in t:
            seq.append(str(len(l)))
        s.append(",".join(seq))
    return "-".join(s)

In [ ]:
find_bumps(trace_gen)

### Frequency analysis

In [ ]:
plot_trace(filter_lowpass(trim(trace_ecdsa, 55_000_000, 56_000_000), 625000000, 250_000_000))

In [ ]:
from scipy.signal  import periodogram
from matplotlib import pyplot as plt

In [ ]:
%matplotlib widget

In [ ]:
f, Pxx_den = periodogram(recenter(trim(trace_ecdsa, 30_000_000, 500_000_000)).samples, 625000000)

In [ ]:
plt.semilogy(f, Pxx_den)
#plt.ylim([1e-7, 1e2])
plt.xlabel('frequency [Hz]')
plt.ylabel('PSD [V**2/Hz]')
plt.show()

In [ ]:
from scipy.signal import find_peaks

In [ ]:
pks = find_peaks(Pxx_den, height=1.52e-7)

In [ ]:
len(pks[0])
for pk in f[pks[0]]:
    print(int(pk))